In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

1. Establish a connection between Python and the Sakila database.

In [59]:
import mysql.connector
import pandas as pd

In [61]:
import getpass
password = getpass.getpass()

 ········


In [10]:
conn = mysql.connector.connect(host='localhost',
                        user='root',
                        passwd=password, database = "sakila")

In [18]:
cursor = conn.cursor()

2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- month: an integer representing the month for which rental data is to be retrieved.
- year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [83]:
def rentals_month(cursor, month, year):
    query = """
    SELECT * FROM sakila.rental 
    WHERE MONTHNAME(rental_date) = %s AND YEAR(rental_date) = %s
    """
    cursor.execute(query, (month, year))
    results = cursor.fetchall()
    columns = ["id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update"]
    return pd.DataFrame(results, columns = columns)

In [85]:
May_rentals = rentals_month(cursor, "May", 2005)

In [97]:
May_rentals

,id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [109]:
grouped = May_rentals.groupby('customer_id')
df = grouped['id'].count()
new_df = pd.DataFrame(df).reset_index() 
new_df.columns = ['customer_id','rentals_05_2005']

3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [113]:
def rental_count_month(df, month, year):
    grouped = df.groupby('customer_id')
    df2 = grouped['id'].count()
    new_df = pd.DataFrame(df2).reset_index() 
    new_df.columns = ['customer_id',f'rentals_{month}_{year}']
    return new_df

In [119]:
rental_count_month(May_rentals,"05",2005)

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [129]:
def compare_rentals(df1, df2):
    df1_grouped = df1.groupby('customer_id')['id'].count().reset_index()
    df1_grouped.columns = ['customer_id', 'rentals_df1']
    df2_grouped = df2.groupby('customer_id')['id'].count().reset_index()
    df2_grouped.columns = ['customer_id', 'rentals_df2']
    df_merge = pd.merge(df1_grouped,df2_grouped,on = 'customer_id', how = 'inner')
    df_merge['difference'] = df_merge['rentals_df1'] - df_merge['rentals_df2']
    return df_merge

In [131]:
compared_df = compare_rentals(May_rentals, May_rentals)
compared_df

,customer_id,rentals_df1,rentals_df2,difference
0,1,2,2,0
1,2,1,1,0
2,3,2,2,0
3,5,3,3,0
4,6,3,3,0
...,...,...,...,...
515,594,4,4,0
516,595,1,1,0
517,596,6,6,0
518,597,2,2,0
